In [ ]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Dataset Creation

In [ ]:
dataset = json.load(open("/content/gdrive/MyDrive/Sem2/IDL/gpt_retriever_eval.json", "r"))


In [ ]:
if 'temperature' not in dataset[1].keys():
  print(dataset[1].keys())

dict_keys(['label', 'num_examples', 'question', 'ta_answer', 'gpt_answer'])


In [ ]:
l = []
l2 = []
for k in dataset:
  l.append([k['label'], k['num_examples'], k['question'], k['ta_answer'], k['gpt_answer']])
len(l)

In [ ]:
df['label'].value_counts()#.keys()

hw3p1        90
hw3          90
other        90
project      90
hw4p1        90
hw4          90
hw3p2        90
quizzes      90
hw2p2        90
hw2          90
hw1p2        90
hw1p1        90
hw1          90
logistics    89
hw4p2        89
lectures     89
hw0          89
Name: label, dtype: int64

In [ ]:
df = pd.DataFrame(l, columns=['label', 'num_examples', 'question', 'ta-answer', 'gpt-answer', 'temperature'])
print(df.shape)
df.head()

(1526, 6)


,label,num_examples,question,ta-answer,gpt-answer,temperature
0,hw0,1,"Hi, I wasn't sure if this is giving away any...",I don't think it is necessary to use np.vector...,"Answer: ""It's hard to say without seeing your ...",0.2
1,hw0,1,I am having trouble understanding why my submi...,Is this the full error message? I don't think ...,"Answer: Unfortunately, without seeing the spec...",0.2
2,hw0,1,"Hi, I'm using slicing the same way that was m...","You should be able to use .astype(), check the...",Answer: It sounds like you might be encounteri...,0.2
3,hw0,1,"""Are we allowed to use for loops after the vec...",It is impossible to do the part on spectrogram...,"Answer: Yes, you are allowed to use for loops ...",0.2
4,hw0,1,How do I submit? 1) ipynb file 2) tar file,hw0p1 requires an *.ipynb file. hw0p2 require...,"Answer: \n\n1) To submit an ipynb file, you ca...",0.2


In [ ]:
df.to_csv("/content/gdrive/MyDrive/Sem2/IDL/gpt_eval_fewshot.csv", index = None)

# Evaluation Script

In [ ]:
def evaluate(df):

  # BARTScore
  bartscore = 0
  bleuscore = 0
  cosinetfidf = 0
  cosinebert = 0
  nli_scores = []
  rows = df.shape[0]
  F1 = 0

  label_mapping = ['contradiction', 'entailment', 'neutral']

  for i in range(df.shape[0]):
    #if i % 10 == 0:
    #  print(i)
    reference = [df.iloc[i, 3]]
    candidate = [df.iloc[i, 4]]
    bartscore += bart_scorer.score(reference, candidate)[0]

    reference = [df.iloc[i, 3].split()]
    candidate = df.iloc[i, 4].split()
    bleuscore += sentence_bleu(reference, candidate, weights=(0.25,0.25,0.25,0.25))

    reference = df.iloc[i, 3]
    candidate = df.iloc[i, 4]
    cosinetfidf += compute_cosine_similarity(reference, candidate)

    reference = sent.encode(df.iloc[i, 3]).reshape(1,-1)
    candidate = sent.encode(df.iloc[i, 4]).reshape(1,-1)
    cosinebert += cosine_similarity(reference, candidate)

    reference = df.iloc[i, 3]
    candidate = df.iloc[i, 4]
    score = nli_model.predict([(reference, candidate)])
    nli_scores.append(label_mapping[score.argmax(axis=1)[0]])
    nli_ = Counter(nli_scores)

    reference = df.iloc[i, 3]
    candidate = df.iloc[i, 4]
    F1 += rouge.get_scores(candidate, reference)[0]['rouge-2']['f']

  return bartscore / rows, bleuscore / rows, cosinetfidf / rows, cosinebert / rows, nli_, F1 / rows

In [ ]:
#for num in labels:
for i in labels:
  tempdf = df[(df['temperature'] ==i)]
  #print("Num: ", num)
  print("i: ", i)
  bartscore, bleuscore, cosinetfidf, cosinebert, nli_, F1 = evaluate(tempdf)
  print("Rows: ", tempdf.shape[0])
  print("Bartscore: ", bartscore)
  print("Bleuscore: ", bleuscore)
  print("cosinetfidf: ", cosinetfidf)
  print("cosinebert: ", cosinebert)
  print("nli_ (entailment): ", nli_['entailment']/tempdf.shape[0])
  print("F1: ", F1)
  print()

i:  0.2
Rows:  170
Bartscore:  -3.59812697242288
Bleuscore:  0.0018568523118162239
cosinetfidf:  0.07423529411764704
cosinebert:  [[0.5443162]]
nli_ (entailment):  0.029411764705882353
F1:  0.018096290435652224

i:  0.0
Rows:  170
Bartscore:  -3.574180970472448
Bleuscore:  0.0017217150613524901
cosinetfidf:  0.07688235294117646
cosinebert:  [[0.5518377]]
nli_ (entailment):  0.052941176470588235
F1:  0.01809628997521239

i:  0.5
Rows:  170
Bartscore:  -3.63977027500377
Bleuscore:  0.0022622976439225645
cosinetfidf:  0.07882352941176467
cosinebert:  [[0.5497767]]
nli_ (entailment):  0.058823529411764705
F1:  0.019264027098137466

i:  0.8
Rows:  170
Bartscore:  -3.742103391535142
Bleuscore:  0.002408000986180716
cosinetfidf:  0.07711764705882354
cosinebert:  [[0.54343665]]
nli_ (entailment):  0.041176470588235294
F1:  0.017167251828445418

i:  1.0
Rows:  170
Bartscore:  -3.817056191668791
Bleuscore:  0.0018783987121598913
cosinetfidf:  0.07152941176470581
cosinebert:  [[0.54723555]]
nli_ 

# BARTScore

In [ ]:
!git clone https://github.com/neulab/BARTScore.git
!pip install -r /content/BARTScore/requirements.txt
!pip install transformers -q

Cloning into 'BARTScore'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 220 (delta 18), reused 14 (delta 14), pack-reused 194
Receiving objects: 100% (220/220), 101.98 MiB | 8.44 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Updating files: 100% (192/192), done.


In [ ]:
ref = "Can you share your implementation of the forward and inference functions?"
cand = '''Without seeing your implementation, it is difficult to determine what might be wrong.However, here are some common mistakes that students make when implementing inference methods:
1. Not loading the saved model weights correctly.
2. Not preprocessing the input data in the same way as during training.
3. Not setting the model to evaluation mode using model.eval().
4. Not converting the model output to the desired format (e.g., probabilities or class labels).
5. Not handling batches correctly (if using batch inference).
6. Not handling GPU/CPU correctly (if using GPU).
You should check your implementation against these potential mistakes and see if any of them apply to your code. Additionally, you can try debugging your code by printing out intermediate values and comparing them to expected values.'''

In [ ]:
from BARTScore.bart_score import BARTScorer
bart_scorer = BARTScorer(device='cuda:0', checkpoint='facebook/bart-large-cnn')
bart_scorer.score([ref], [cand], batch_size=4)

[-4.028334140777588]

# BLEU

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
#s = "this is a test"
reference = [ref.split()]
candidate = cand.split()
print(reference)
print(candidate)
score = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0))
print(score)

[['Can', 'you', 'share', 'your', 'implementation', 'of', 'the', 'forward', 'and', 'inference', 'functions?']]
['Without', 'seeing', 'your', 'implementation,', 'it', 'is', 'difficult', 'to', 'determine', 'what', 'might', 'be', 'wrong.However,', 'here', 'are', 'some', 'common', 'mistakes', 'that', 'students', 'make', 'when', 'implementing', 'inference', 'methods:', '1.', 'Not', 'loading', 'the', 'saved', 'model', 'weights', 'correctly.', '2.', 'Not', 'preprocessing', 'the', 'input', 'data', 'in', 'the', 'same', 'way', 'as', 'during', 'training.', '3.', 'Not', 'setting', 'the', 'model', 'to', 'evaluation', 'mode', 'using', 'model.eval().', '4.', 'Not', 'converting', 'the', 'model', 'output', 'to', 'the', 'desired', 'format', '(e.g.,', 'probabilities', 'or', 'class', 'labels).', '5.', 'Not', 'handling', 'batches', 'correctly', '(if', 'using', 'batch', 'inference).', '6.', 'Not', 'handling', 'GPU/CPU', 'correctly', '(if', 'using', 'GPU).', 'You', 'should', 'check', 'your', 'implementation',

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


# Cosine Similarity

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

sentence1 = 'I like eating ice cream on a hot summer day.'
sentence2 = 'Only boring people do not like eating ice cream.'
sentence3 = 'I do not like going out during the summer since it is so hot.'

# returns the cosine similarity value of the two given texts
def compute_cosine_similarity(text1, text2):
    
    # stores text in a list
    list_text = [text1, text2]
    
    # converts text into vectors with the TF-IDF 
    vectorizer = TfidfVectorizer(stop_words='english')
    vectorizer.fit_transform(list_text)
    tfidf_text1, tfidf_text2 = vectorizer.transform([list_text[0]]), vectorizer.transform([list_text[1]])
    
    # computes the cosine similarity
    cs_score = cosine_similarity(tfidf_text1, tfidf_text2)
    
    return np.round(cs_score[0][0],2)

# Sentence BERT

In [ ]:
!pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
sent = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
reference = sent.encode(ref).reshape(1,-1)
candidate = sent.encode(cand).reshape(1,-1)
score = cosine_similarity(reference, candidate)
score

array([[0.28064072]], dtype=float32)

# NLI Models

In [ ]:
from sentence_transformers import CrossEncoder
nli_model = CrossEncoder('cross-encoder/nli-roberta-base')
scores = nli_model.predict([('A man is eating pizza', 'A man eats something'), ('A black race car starts up in front of a crowd of people.', 'A man is driving down a lonely road.')])

#Convert scores to labels
label_mapping = ['contradiction', 'entailment', 'neutral']
labels = [label_mapping[score_max] for score_max in scores.argmax(axis=1)]

In [ ]:
reference = ref
candidate = cand
score = nli_model.predict([(reference, candidate)])
print("ref:", reference)
print("cand:", candidate)
score.argmax(axis=1)

ref: Can you share your implementation of the forward and inference functions?
cand: Without seeing your implementation, it is difficult to determine what might be wrong.However, here are some common mistakes that students make when implementing inference methods:
1. Not loading the saved model weights correctly.
2. Not preprocessing the input data in the same way as during training.
3. Not setting the model to evaluation mode using model.eval().
4. Not converting the model output to the desired format (e.g., probabilities or class labels).
5. Not handling batches correctly (if using batch inference).
6. Not handling GPU/CPU correctly (if using GPU).
You should check your implementation against these potential mistakes and see if any of them apply to your code. Additionally, you can try debugging your code by printing out intermediate values and comparing them to expected values.


array([2])

In [ ]:
from collections import Counter
Counter(bleu_scores_labels)

Counter({'entailment': 54, 'contradiction': 132})

# BERTScore

In [ ]:
!pip install bert_score --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from bert_score import score

bleu_scores = []

for i in range(df.shape[0]):
  if i % 10 == 0:
    print(i)
  reference = df.iloc[i, 1]
  candidate = df.iloc[i, 2]
  P, R, F1 = score([reference], [candidate], lang="en", device=device)
  bleu_scores.append(F1)

df['BERTScore_F1'] = bleu_scores
print(np.average(bleu_scores))

# Rouge

In [ ]:
!pip install rouge --quiet


In [ ]:
from rouge import Rouge

model_out = cand #"he began by starting a five person war cabinet and included chamberlain as lord president of the council"

reference = ref #"he began his premiership by forming a five-man war cabinet which included chamberlain as lord president of the council"

rouge = Rouge()

result = rouge.get_scores(model_out, reference)

In [ ]:
result[0]['rouge-2']['f']

0.015037592594267754